In [14]:
import pandas as pd
import gurobipy as gb
from gurobipy import GRB

# Load datasets
welders_data = pd.read_csv("/Users/huiyisang/Desktop/welders_data.csv")
welders_speed_data = pd.read_csv("/Users/huiyisang/Desktop/welders_speed_data.csv")

# Merge datasets on Welder_ID if necessary
if "Welder_ID" in welders_speed_data.columns:
    welders_data = welders_data.merge(welders_speed_data, on="Welder_ID", how="left")

# Extract relevant columns
welders_data = welders_data[["Welder_ID", "Safety_Rating", "Speed_Rating", 
                             "SMAW_Proficient", "GMAW_Proficient", 
                             "FCAW_Proficient", "GTAW_Proficient", "Experience_10_Years"]]

# Number of welders to hire
num_welders_to_hire = 16
num_welders = len(welders_data)

# Create Gurobi model
model = gb.Model("Welder Selection")

# Decision variables: Relaxed continuous variables between 0 and 1
x = model.addVars(num_welders, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="WelderSelection")

# Objective function: Maximize the average speed rating
model.setObjective(gb.quicksum(welders_data.iloc[i]["Speed_Rating"] * x[i] for i in range(num_welders)) / num_welders_to_hire, GRB.MAXIMIZE)

# Constraints
# 1. Select exactly 16 welders
model.addConstr(gb.quicksum(x[i] for i in range(num_welders)) == num_welders_to_hire, "TotalWelders")

# 2. At least 50% of hired welders must be proficient in both SMAW and GMAW
model.addConstr(gb.quicksum(x[i] for i in range(num_welders) if welders_data.iloc[i]["SMAW_Proficient"] == 1 and welders_data.iloc[i]["GMAW_Proficient"] == 1) >= 0.5 * num_welders_to_hire, "SMAW_GMAW_Proficiency")

# 3. At least two welders proficient in each welding technique
for technique in ["SMAW_Proficient", "GMAW_Proficient", "FCAW_Proficient", "GTAW_Proficient"]:
    model.addConstr(gb.quicksum(x[i] for i in range(num_welders) if welders_data.iloc[i][technique] == 1) >= 2, f"{technique}_Requirement")

# 4. At least 30% of hired welders must have more than 10 years of experience
model.addConstr(gb.quicksum(x[i] for i in range(num_welders) if welders_data.iloc[i]["Experience_10_Years"] == 1) >= 0.3 * num_welders_to_hire, "ExperienceRequirement")

# 5. The average safety rating of all hired welders must be at least 3.9
model.addConstr(gb.quicksum(welders_data.iloc[i]["Safety_Rating"] * x[i] for i in range(num_welders)) / num_welders_to_hire >= 3.9, "SafetyRequirement")

# 6. The average speed rating must be at least 3.1
model.addConstr(gb.quicksum(welders_data.iloc[i]["Speed_Rating"] * x[i] for i in range(num_welders)) / num_welders_to_hire >= 3.1, "SpeedRequirement")

# 7. At least three welders must have a speed rating of 1 and a safety rating of 3 or 4
model.addConstr(gb.quicksum(x[i] for i in range(num_welders) if welders_data.iloc[i]["Speed_Rating"] == 1 and welders_data.iloc[i]["Safety_Rating"] >= 3) >= 3, "LowSpeedHighSafety")

# 8. The number of welders hired from range 70-100 must be at least one more than twice the number hired from range 101-130
num_hired_70_100 = gb.quicksum(x[i] for i in range(69, 100))
num_hired_101_130 = gb.quicksum(x[i] for i in range(100, 130))
model.addConstr(num_hired_70_100 >= 2 * num_hired_101_130 + 1, "HiringRange")

# Solve the relaxed LP problem
model.optimize()

# Print results
print("Optimal objective function value (Linear Relaxation):", model.objVal)

relax_model = model.relax()
relax_model.optimize()
print("Optimal objective function value (Linear Relaxation):", relax_model.objVal)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11 rows, 144 columns and 905 nonzeros
Model fingerprint: 0x09b3986a
Coefficient statistics:
  Matrix range     [6e-02, 2e+00]
  Objective range  [6e-02, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 0 rows and 9 columns
Presolve time: 0.00s
Presolved: 11 rows, 135 columns, 850 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.6875000e+00   7.190000e+01   0.000000e+00      0s
      24    3.1625000e+00   0.000000e+00   0.000000e+00      0s

Solved in 24 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.162500000e+00
Optimal objective function value (Linear Relaxation): 3.1625
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread co